<a href="https://colab.research.google.com/github/thenewgen58-web/my-fastapi-backend/blob/main/Copy_of_volleyball_trajectory_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Installeer benodigde tools
!pip install -q flask pyngrok requests

import os
import json
import requests
from flask import Flask, request
from pyngrok import ngrok
from threading import Thread

# 1. Start ngrok om een publieke URL te creëren
# Je moet een ngrok authtoken instellen voor langdurig gebruik
ngrok.set_auth_token("35yNOuzdqPvoXGmXKHyS4qkAm1r_56w1C8XvfJB4EjKiCVcw7")
public_url = ngrok.connect(5000).public_url
print(f" * Colab Webhook URL: {public_url}")
# Plak deze URL in Make.com Scenario 1 (Stap 2.2)

# 2. Definieer de Flask App
app = Flask(__name__)

@app.route('/analyze', methods=['POST'])
def analyze_video():
    # Ontvang de data van Make.com
    data = request.get_json()
    video_url = data.get('video_url')
    upload_id = data.get('upload_id')
    callback_url = data.get('callback_url')

    if not video_url or not upload_id or not callback_url:
        return json.dumps({"status": "error", "message": "Missing parameters"}), 400

    # Start de analyse in een aparte thread om de webhook snel te beantwoorden
    Thread(target=run_analysis_script, args=(video_url, upload_id, callback_url)).start()

    return json.dumps({"status": "analysis_started", "id": upload_id}), 200

# 3. Start de Flask Server
def run_flask():
    app.run(port=5000)
https://stoutish-jae-hardheadedly.ngrok-free.dev/
Thread(target=run_flask).start()

 * Colab Webhook URL: https://stoutish-jae-hardheadedly.ngrok-free.dev


# Volleyball Trajectory Tracker with Millisecond Precision

This notebook provides a complete solution for tracking volleyball trajectories with millisecond-precision timestamps.

## Features

1. **Frame Extraction**: Extract frames at exactly 30 FPS with millisecond timestamps
2. **Ball Detection**: Use Roboflow model to detect volleyball in each frame
3. **Coordinate Tracking**: Record ball coordinates with precise timestamps
4. **Trajectory Visualization**: Draw trajectory line on video/image
5. **Database Storage**: Save all data to SQLite database

## Requirements

### Google Colab Secrets

Add these secrets in Colab (🔑 icon in left sidebar):
1. **ROBOFLOW_API_KEY** - Your Roboflow API key
2. **GOOGLE_CREDENTIALS** - Full content of your credentials.json file (optional, only if using Google Drive)

## 1. Install Dependencies

In [ ]:
!pip install -q roboflow opencv-python numpy pillow
!pip install -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 61.7 MB/s eta 0:00:00


## 2. Import Libraries

In [ ]:
import os
import cv2
import numpy as np
import sqlite3
import json
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
from roboflow import Roboflow
from PIL import Image
import io

# Import Google Colab userdata for secrets
try:
    from google.colab import userdata
    IN_COLAB = True
    print("✓ Google Colab detected")
except ImportError:
    IN_COLAB = False
    print("⚠️ Not in Google Colab - set API keys manually")

print("✓ All libraries imported successfully")

✓ Google Colab detected
✓ All libraries imported successfully


## 3. Get API Keys from Colab Secrets

In [ ]:
# Get Roboflow API key
if IN_COLAB:
    try:
        ROBOFLOW_API_KEY = userdata.get('API')
        print("✓ Roboflow API key retrieved from Colab secrets")
    except Exception as e:
        print(f"⚠️ Could not retrieve Roboflow API key: {e}")
        print("Add 'ROBOFLOW_API_KEY' to Colab secrets (🔑 icon in sidebar)")
        ROBOFLOW_API_KEY = None
else:
    # For local use
    ROBOFLOW_API_KEY = [API]  # Replace with your key
    print("⚠️ Using manually set API key (not in Colab)")

✓ Roboflow API key retrieved from Colab secrets


## 4. VolleyballTrajectoryTracker Class

In [ ]:
class VolleyballTrajectoryTracker:
    """
    Complete solution for tracking volleyball trajectory with millisecond precision.
    """

    def __init__(self, roboflow_api_key: str, model_id: str, db_path: str = "volleyball_tracking.db"):
        """
        Initialize the tracker.

        Args:
            roboflow_api_key: Roboflow API key
            model_id: Roboflow model ID (format: workspace/project/version)
            db_path: Path to SQLite database file
        """
        self.roboflow_api_key = roboflow_api_key # Corrected: use passed argument
        self.model_id = model_id         # Corrected: use passed argument
        self.db_path = db_path
        self.model = None                # Initialize model to None, it will be set by _init_model

        # Initialize Roboflow model
        self._init_model()

        # Initialize database
        self._init_database()

    def _init_model(self):
        """Initialize Roboflow model."""
        try:
            rf = Roboflow(api_key=self.roboflow_api_key)

            # Parse model ID
            parts = self.model_id.split('/')
            if len(parts) != 3:
                raise ValueError(f"Invalid model ID format. Expected 'workspace/project/version', got '{self.model_id}'")

            workspace_id, project_id, version = parts

            # Get project and model
            project = rf.workspace(workspace_id).project(project_id)
            self.model = project.version(int(version)).model

            print(f"✓ Roboflow model initialized: {self.model_id}")
        except Exception as e:
            print(f"✗ Error initializing Roboflow model: {e}")
            raise

    def _init_database(self):
        """Initialize SQLite database with required tables."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Create videos table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS videos (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                filename TEXT NOT NULL,
                fps REAL NOT NULL,
                width INTEGER NOT NULL,
                height INTEGER NOT NULL,
                total_frames INTEGER NOT NULL,
                duration_ms REAL NOT NULL,
                processed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')

        # Create detections table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS detections (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                video_id INTEGER NOT NULL,
                frame_number INTEGER NOT NULL,
                timestamp_ms REAL NOT NULL,
                center_x REAL NOT NULL,
                center_y REAL NOT NULL,
                bbox_x1 REAL NOT NULL,
                bbox_y1 REAL NOT NULL,
                bbox_x2 REAL NOT NULL,
                bbox_y2 REAL NOT NULL,
                confidence REAL NOT NULL,
                class_name TEXT NOT NULL,
                FOREIGN KEY (video_id) REFERENCES videos (id)
            )
        ''')

        # Create index for faster queries
        cursor.execute('''
            CREATE INDEX IF NOT EXISTS idx_video_frame
            ON detections (video_id, frame_number)
        ''')

        conn.commit()
        conn.close()
        print(f"✓ Database initialized: {self.db_path}")

    def extract_frames_with_timestamps(self, video_path: str, target_fps: int = 30,
                                      output_dir: str = "frames") -> Tuple[List[str], List[float], Dict]:
        """
        Extract frames from video at specified FPS with millisecond timestamps.

        Args:
            video_path: Path to input video
            target_fps: Target FPS for frame extraction (default: 30)
            output_dir: Directory to save extracted frames

        Returns:
            Tuple of (frame_paths, timestamps_ms, video_info)
        """
        print(f"\nExtracting frames from: {video_path}")
        print(f"Target FPS: {target_fps}")

        # Create output directory
        os.makedirs(output_dir, exist_ok=True)

        # Open video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")

        # Get video properties
        original_fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration_ms = (total_frames / original_fps) * 1000

        video_info = {
            'filename': os.path.basename(video_path),
            'original_fps': original_fps,
            'target_fps': target_fps,
            'width': width,
            'height': height,
            'total_frames': total_frames,
            'duration_ms': duration_ms
        }

        print(f"Video properties: {width}x{height}, {original_fps:.2f} FPS, {total_frames} frames, {duration_ms:.2f} ms")

        # Calculate frame interval
        frame_interval = original_fps / target_fps
        ms_per_frame = 1000.0 / target_fps

        frame_paths = []
        timestamps_ms = []
        frame_count = 0
        extracted_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Check if this frame should be extracted
            if frame_count % int(frame_interval) == 0 or frame_interval < 1:
                # Calculate precise timestamp in milliseconds
                timestamp_ms = extracted_count * ms_per_frame

                # Save frame with timestamp in filename
                frame_filename = f"frame_{extracted_count:06d}_{timestamp_ms:.3f}ms.jpg"
                frame_path = os.path.join(output_dir, frame_filename)
                cv2.imwrite(frame_path, frame)

                frame_paths.append(frame_path)
                timestamps_ms.append(timestamp_ms)
                extracted_count += 1

                if extracted_count % 30 == 0:
                    print(f"Extracted: {extracted_count} frames")

            frame_count += 1

        cap.release()
        print(f"✓ Extracted {extracted_count} frames at {target_fps} FPS")

        return frame_paths, timestamps_ms, video_info

    def detect_ball_in_frames(self, frame_paths: List[str], timestamps_ms: List[float],
                             confidence_threshold: float = 0.4) -> List[Dict]:
        """
        Detect volleyball in each frame using Roboflow model.

        Args:
            frame_paths: List of frame file paths
            timestamps_ms: List of timestamps in milliseconds
            confidence_threshold: Minimum confidence for detection

        Returns:
            List of detection dictionaries
        """
        print(f"\nDetecting volleyball in {len(frame_paths)} frames...")
        print(f"Confidence threshold: {confidence_threshold}")

        detections = []

        for idx, (frame_path, timestamp_ms) in enumerate(zip(frame_paths, timestamps_ms)):
            try:
                # Run inference
                result = self.model.predict(frame_path, confidence=confidence_threshold).json()

                # Process predictions
                if 'predictions' in result and len(result['predictions']) > 0:
                    # Take the highest confidence detection
                    pred = max(result['predictions'], key=lambda x: x['confidence'])

                    # Calculate center coordinates
                    center_x = pred['x']
                    center_y = pred['y']
                    width = pred['width']
                    height = pred['height']

                    # Calculate bounding box
                    bbox_x1 = center_x - width / 2
                    bbox_y1 = center_y - height / 2
                    bbox_x2 = center_x + width / 2
                    bbox_y2 = center_y + height / 2

                    detection = {
                        'frame_number': idx,
                        'frame_path': frame_path,
                        'timestamp_ms': timestamp_ms,
                        'center_x': center_x,
                        'center_y': center_y,
                        'bbox_x1': bbox_x1,
                        'bbox_y1': bbox_y1,
                        'bbox_x2': bbox_x2,
                        'bbox_y2': bbox_y2,
                        'confidence': pred['confidence'],
                        'class_name': pred['class']
                    }

                    detections.append(detection)

                if (idx + 1) % 30 == 0:
                    print(f"Processed: {idx + 1}/{len(frame_paths)} frames, {len(detections)} detections")

            except Exception as e:
                print(f"Warning: Error processing frame {idx}: {e}")
                continue

        print(f"✓ Detection complete: {len(detections)} balls detected in {len(frame_paths)} frames")
        return detections

    def save_to_database(self, video_info: Dict, detections: List[Dict]) -> int:
        """
        Save video info and detections to database.

        Args:
            video_info: Dictionary with video information
            detections: List of detection dictionaries

        Returns:
            Video ID in database
        """
        print(f"\nSaving to database: {self.db_path}")

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Insert video record
        cursor.execute('''
            INSERT INTO videos (filename, fps, width, height, total_frames, duration_ms)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            video_info['filename'],
            video_info['target_fps'],
            video_info['width'],
            video_info['height'],
            video_info['total_frames'],
            video_info['duration_ms']
        ))

        video_id = cursor.lastrowid

        # Insert detection records
        for det in detections:
            cursor.execute('''
                INSERT INTO detections
                (video_id, frame_number, timestamp_ms, center_x, center_y,
                 bbox_x1, bbox_y1, bbox_x2, bbox_y2, confidence, class_name)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                video_id,
                det['frame_number'],
                det['timestamp_ms'],
                det['center_x'],
                det['center_y'],
                det['bbox_x1'],
                det['bbox_y1'],
                det['bbox_x2'],
                det['bbox_y2'],
                det['confidence'],
                det['class_name']
            ))

        conn.commit()
        conn.close()

        print(f"✓ Saved video ID {video_id} with {len(detections)} detections")
        return video_id

    def visualize_trajectory(self, video_path: str, detections: List[Dict],
                            output_path: str = "trajectory_video.mp4",
                            output_image_path: str = "trajectory_image.jpg",
                            line_color: Tuple[int, int, int] = (0, 255, 0),
                            line_thickness: int = 2,
                            point_radius: int = 5) -> Tuple[str, str]:
        """
        Create video and image with trajectory visualization.

        Args:
            video_path: Path to original video
            detections: List of detection dictionaries
            output_path: Path for output video
            output_image_path: Path for output image
            line_color: RGB color for trajectory line
            line_thickness: Thickness of trajectory line
            point_radius: Radius of detection points

        Returns:
            Tuple of (video_path, image_path)
        """
        print(f"\nVisualizing trajectory...")

        if len(detections) == 0:
            print("⚠️ No detections to visualize")
            return None, None

        # Open video
        cap = cv2.VideoCapture(video_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        # Extract trajectory points
        trajectory_points = [(int(det['center_x']), int(det['center_y'])) for det in detections]

        # Create trajectory image (static)
        ret, first_frame = cap.read()
        if ret:
            trajectory_image = first_frame.copy()

            # Draw trajectory line
            for i in range(len(trajectory_points) - 1):
                cv2.line(trajectory_image, trajectory_points[i], trajectory_points[i + 1],
                        line_color, line_thickness)

            # Draw points
            for point in trajectory_points:
                cv2.circle(trajectory_image, point, point_radius, (0, 0, 255), -1)



        # Reset video capture
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        # Create video with progressive trajectory
        frame_idx = 0
        detection_dict = {det['frame_number']: det for det in detections}

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Draw trajectory up to current frame
            current_trajectory = [trajectory_points[i] for i in range(len(trajectory_points))
                                 if detections[i]['frame_number'] <= frame_idx]

            for i in range(len(current_trajectory) - 1):
                cv2.line(frame, current_trajectory[i], current_trajectory[i + 1],
                        line_color, line_thickness)

            # Draw current detection
            if frame_idx in detection_dict:
                det = detection_dict[frame_idx]
                center = (int(det['center_x']), int(det['center_y']))
                cv2.circle(frame, center, point_radius, (0, 0, 255), -1)

                # Add label
                label = f"{det['timestamp_ms']:.1f}ms"
                cv2.putText(frame, label, (center[0] + 10, center[1] - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            out.write(frame)
            frame_idx += 1

        cap.release()
        out.release()

        print(f"✓ Trajectory video saved: {output_path}")
        return output_path, output_image_path

    def process_video(self, video_path: str, target_fps: int = 30,
                     confidence_threshold: float = 0.4,
                     output_video: str = "trajectory_video.mp4",
                     output_image: str = "trajectory_image.jpg",
                     cleanup_frames: bool = True) -> Dict:
        """
        Complete pipeline: extract frames, detect ball, save to database, visualize.

        Args:
            video_path: Path to input video
            target_fps: Target FPS for frame extraction
            confidence_threshold: Minimum confidence for detection
            output_video: Path for output video
            output_image: Path for output image
            cleanup_frames: Delete extracted frames after processing

        Returns:
            Dictionary with processing results
        """
        print("="*60)
        print("VOLLEYBALL TRAJECTORY TRACKING PIPELINE")
        print("="*60)

        results = {}
        frames_dir = "frames"

        try:
            # Step 1: Extract frames
            frame_paths, timestamps_ms, video_info = self.extract_frames_with_timestamps(
                video_path, target_fps, frames_dir
            )
            results['frame_count'] = len(frame_paths)
            results['video_info'] = video_info

            # Step 2: Detect ball
            detections = self.detect_ball_in_frames(frame_paths, timestamps_ms, confidence_threshold)
            results['detection_count'] = len(detections)
            results['detections'] = detections

            # Step 3: Save to database
            video_id = self.save_to_database(video_info, detections)
            results['video_id'] = video_id

            # Step 4: Visualize trajectory
            video_out, image_out = self.visualize_trajectory(
                video_path, detections, output_video, output_image
            )
            results['output_video'] = video_out
            results['output_image'] = image_out

            print("\n" + "="*60)
            print("✓ PIPELINE COMPLETED SUCCESSFULLY")
            print("="*60)
            print(f"Frames extracted: {results['frame_count']}")
            print(f"Balls detected: {results['detection_count']}")
            print(f"Database video ID: {results['video_id']}")
            print(f"Output video: {results['output_video']}")
            print(f"Output image: {results['output_image']}")

            return results

        finally:
            # Cleanup frames
            if cleanup_frames and os.path.exists(frames_dir):
                import shutil
                try:
                    shutil.rmtree(frames_dir)
                    print(f"\n✓ Cleaned up frames directory: {frames_dir}")
                except Exception as e:
                    print(f"⚠️ Could not clean up frames: {e}")

    def export_detections_to_json(self, video_id: int, output_path: str = "detections.json"):
        """
        Export detections from database to JSON file.

        Args:
            video_id: Video ID in database
            output_path: Path for output JSON file
        """
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Get video info
        cursor.execute('SELECT * FROM videos WHERE id = ?', (video_id,))
        video_row = cursor.fetchone()

        # Get detections
        cursor.execute('SELECT * FROM detections WHERE video_id = ? ORDER BY frame_number', (video_id,))
        detection_rows = cursor.fetchall()

        conn.close()

        # Format data
        data = {
            'video': {
                'id': video_row[0],
                'filename': video_row[1],
                'fps': video_row[2],
                'width': video_row[3],
                'height': video_row[4],
                'total_frames': video_row[5],
                'duration_ms': video_row[6],
                'processed_at': video_row[7]
            },
            'detections': [
                {
                    'frame_number': row[2],
                    'timestamp_ms': row[3],
                    'center_x': row[4],
                    'center_y': row[5],
                    'bbox': {
                        'x1': row[6],
                        'y1': row[7],
                        'x2': row[8],
                        'y2': row[9]
                    },
                    'confidence': row[10],
                    'class': row[11]
                }
                for row in detection_rows
            ]
        }

        with open(output_path, 'w') as f:
            json.dump(data, f, indent=2)

        print(f"✓ Exported {len(detection_rows)} detections to: {output_path}")

print("✓ VolleyballTrajectoryTracker class defined")

✓ VolleyballTrajectoryTracker class defined


## 5. Configuration

**⚠️ IMPORTANT: Update these values with your own data!**

In [ ]:
# Roboflow configuration
# Model ID format: workspace/project/version
ROBOFLOW_MODEL_ID = "understanding-ai/the-ball-of-all-balls-xccli/6"  # ⚠️ REPLACE with your model ID

# Video configuration
VIDEO_PATH = "volleyball_video.mp4"  # ⚠️ REPLACE with your video path
TARGET_FPS = 30  # Extract frames at 30 FPS
CONFIDENCE_THRESHOLD = 0.4  # Minimum confidence for ball detection

# Output configuration
OUTPUT_VIDEO = "trajectory_video.mp4"
OUTPUT_IMAGE = "trajectory_image.jpg"
DATABASE_PATH = "volleyball_tracking.db"

print("✓ Configuration set")
print(f"  Model ID: {ROBOFLOW_MODEL_ID}")
print(f"  Video path: {VIDEO_PATH}")
print(f"  Target FPS: {TARGET_FPS}")
print(f"  API Key present: {ROBOFLOW_API_KEY is not None}")

✓ Configuration set
  Model ID: understanding-ai/the-ball-of-all-balls-xccli/6
  Video path: volleyball_video.mp4
  Target FPS: 30
  API Key present: True


## 6. Initialize Tracker

In [ ]:
# Create tracker object
tracker = VolleyballTrajectoryTracker(
    roboflow_api_key=ROBOFLOW_API_KEY,
    model_id=ROBOFLOW_MODEL_ID
)
print("✓ Tracker initialized")

loading Roboflow workspace...
loading Roboflow project...
✓ Roboflow model initialized: understanding-ai/the-ball-of-all-balls-xccli/6
✓ Database initialized: volleyball_tracking.db
✓ Tracker initialized


## 7. Run Complete Pipeline

This cell runs all steps:
1. Extract frames at 30 FPS with millisecond timestamps
2. Detect volleyball in each frame
3. Save all data to database
4. Create trajectory visualization (video + image)
5. Cleanup temporary files

In [ ]:
try:
    results = tracker.process_video(
        video_path=VIDEO_PATH,
        target_fps=TARGET_FPS,
        confidence_threshold=CONFIDENCE_THRESHOLD,
        output_video=OUTPUT_VIDEO,
        output_image=OUTPUT_IMAGE,
        cleanup_frames=True
    )

    print(f"\n🎉 Success! Results:")
    print(f"  Video ID: {results['video_id']}")
    print(f"  Detections: {results['detection_count']}")
    print(f"  Output video: {results['output_video']}")
    print(f"  Output image: {results['output_image']}")

except Exception as e:
    print(f"\n❌ Pipeline failed: {e}")
    import traceback
    traceback.print_exc()

VOLLEYBALL TRAJECTORY TRACKING PIPELINE

Extracting frames from: volleyball_video.mp4
Target FPS: 30
Video properties: 1920x1080, 30.00 FPS, 90 frames, 3000.00 ms
Extracted: 30 frames
Extracted: 60 frames
Extracted: 90 frames
✓ Extracted 90 frames at 30 FPS

Detecting volleyball in 90 frames...
Confidence threshold: 0.4


## 8. Export Detections to JSON

In [ ]:
# Export detections to JSON file
# Replace video_id with the ID from previous step
tracker.export_detections_to_json(video_id=results['video_id'], output_path="detections.json")

✓ Exported 74 detections to: detections.json


## 9. Query Database

Example queries to explore the data:

In [ ]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect(DATABASE_PATH)

# Query all videos
videos_df = pd.read_sql_query("SELECT * FROM videos", conn)
print("Videos:")
print(videos_df)

# Query detections for latest video
detections_df = pd.read_sql_query("""
    SELECT frame_number, timestamp_ms, center_x, center_y, confidence, class_name
    FROM detections
    WHERE video_id = (SELECT MAX(id) FROM videos)
    ORDER BY frame_number
""", conn)

print("\nDetections:")
print(detections_df.head(10))
print(f"\nTotal detections: {len(detections_df)}")

conn.close()

Videos:
   id              filename   fps  width  height  total_frames  duration_ms  \
0   1  volleyball_video.mp4  30.0   1920    1080            88  2933.333333   

          processed_at  
0  2025-11-22 15:17:47  

Detections:
   frame_number  timestamp_ms     center_x    center_y  confidence  class_name
0             0      0.000000  1112.288513  164.103928    0.832488  volleyball
1             1     33.333333  1117.268799  161.151970    0.788801  volleyball
2             2     66.666667  1122.967041  158.868141    0.798167  volleyball
3             3    100.000000  1131.043335  157.092133    0.786288  volleyball
4             4    133.333333  1138.052612  155.904121    0.747290  volleyball
5             5    166.666667  1143.008850  157.513847    0.734516  volleyball
6             6    200.000000  1149.592346  157.640923    0.720875  volleyball
7             7    233.333333  1153.678162  160.289749    0.766963  volleyball
8             8    266.666667  1160.840576  162.481544    0

## 10. Alternative: Step-by-Step Execution

If you want more control, you can run each step separately:

In [ ]:
# Step 1: Extract frames
frame_paths, timestamps_ms, video_info = tracker.extract_frames_with_timestamps(
    VIDEO_PATH, target_fps=TARGET_FPS
)

In [ ]:
# Step 2: Detect ball
detections = tracker.detect_ball_in_frames(frame_paths, timestamps_ms, CONFIDENCE_THRESHOLD)

In [ ]:
# Step 3: Save to database
video_id = tracker.save_to_database(video_info, detections)
print(f"Video ID: {video_id}")

In [ ]:
# Step 4: Visualize trajectory
output_video, output_image = tracker.visualize_trajectory(
    VIDEO_PATH, detections, OUTPUT_VIDEO, OUTPUT_IMAGE
)

## 📝 Notes

### Key Features

1. **Millisecond Precision**: Timestamps are calculated with millisecond precision (e.g., 33.333ms, 66.667ms, 100.000ms for 30 FPS)
2. **Database Storage**: All data is stored in SQLite database with proper indexing
3. **Trajectory Visualization**: Creates both video (progressive) and image (complete trajectory)
4. **JSON Export**: Export detections to JSON for further analysis

### Database Schema

**videos table:**
- id, filename, fps, width, height, total_frames, duration_ms, processed_at

**detections table:**
- id, video_id, frame_number, timestamp_ms, center_x, center_y, bbox_x1, bbox_y1, bbox_x2, bbox_y2, confidence, class_name

### Timestamp Calculation

For 30 FPS:
- Frame 0: 0.000 ms
- Frame 1: 33.333 ms
- Frame 2: 66.667 ms
- Frame 3: 100.000 ms
- etc.

Formula: `timestamp_ms = frame_number * (1000 / fps)`

### Troubleshooting

**No detections:**
- Lower confidence_threshold (try 0.2 or 0.3)
- Check if your model is trained for volleyball detection
- Verify video quality and ball visibility

**Memory issues:**
- Set cleanup_frames=True to delete frames after processing
- Process shorter video clips
- Reduce target_fps

**Model errors:**
- Verify model ID format: workspace/project/version
- Check API key permissions
- Ensure model is deployed and accessible

ALS HET NIET WERKT VERWIJDER ALLES ONDER DEZE TEXT BOX

In [ ]:
!pip install modal


In [ ]:
!modal setup  # Authenticate with your Modal account

Was not able to launch web browser
Please go to this URL manually and complete the flow:

]8;id=106098;https://modal.com/token-flow/tf-H94rtBxmx9TjQLtVL6tThQ\https://modal.com/token-flow/tf-H94rtBxmx9TjQLtVL6tThQ]8;;\

⠋ Waiting for authentication in the web browser
⠋ Waiting for token flow to complete...


In [ ]:
%%writefile video_api.py
import modal

# Define your image with dependencies
image = modal.Image.debian_slim().pip_install(
    "numpy", "opencv-python", "torch"  # Add your dependencies here
)

app = modal.App("video-processor", image=image)

@app.function(gpu="T4")  # Remove gpu param if not needed
@modal.fastapi_endpoint(method="POST")
def process_video(video_url: str, video_id: str):
    # Your existing processing logic here
    # Download video from video_url, process it, return results

    result = {
        "video_id": video_id,
        "status": "completed",
        "data": {"your": "results"}
    }
    return result


Overwriting video_api.py


In [ ]:
!modal deploy video_api.py

⠸ Creating objects...
⠦ Creating objects...
├── 🔨 Created mount /content/video_api.py
⠧ Creating objects...
├── 🔨 Created mount /content/video_api.py
└── ⠋ Creating function process_video...
╭─ Error ──────────────────────────────────────────────────────────────────────╮
│ Web endpoint Functions require `FastAPI` to be installed in their Image.     │
│ This used to happen automatically, but it must now be done explicitly. We    │
│ recommend installing `fastapi` to include useful extensions.                 │
╰──────────────────────────────────────────────────────────────────────────────╯


In [ ]:
%%writefile video_api.py
import modal

# Define your image with dependencies
image = modal.Image.debian_slim().pip_install(
    "numpy", "opencv-python", "torch", "fastapi"  # Add your dependencies here
)

app = modal.App("video-processor", image=image)

@app.function(gpu="T4")  # Remove gpu param if not needed
@modal.fastapi_endpoint(method="POST")
def process_video(video_url: str, video_id: str):
    # Your existing processing logic here
    # Download video from video_url, process it, return results

    result = {
        "video_id": video_id,
        "status": "completed",
        "data": {"your": "results"}
    }
    return result

Overwriting video_api.py


In [ ]:
!modal deploy video_api.py

⠸ Creating objects...
Building image im-e3nGujv93gdAx6Ewxp9w7D
⠦ Creating objects...
⠦ Creating objects...
⠏ Creating objects...

=> Step 0: FROM base
⠋ Creating objects...

=> Step 1: RUN python -m pip install fastapi numpy opencv-python torch
⠋ Creating objects...
⠹ Creating objects...
⠴ Creating objects...
⠇ Creating objects...
⠙ Creating objects...
⠴ Creating objects...
Looking in indexes: http://pypi-mirror.modal.local:5555/simple
⠧ Creating objects...
⠇ Creating objects...
⠏ Creating objects...
⠏ Creating objects...
⠙ Creating objects...
⠸ Creating objects...
⠼ Creating objects...
⠼ Creating objects...
⠦ Creating objects...
⠦ Creating objects...
⠧ Creating objects...
⠧ Creating objects...
⠏ Creating objects...
⠋ Creating objects...
⠋ Creating objects...
⠋ Creating objects...
⠋ Creating objects...
⠸ Creating objects...
⠧ Creating objects...
⠋ Creating objects...
⠸ Creating objects...
⠴ Creating objects...
⠴ Creating objects...
⠦ Creating objects...
⠦ Creating objects...
⠧ Creating

In [ ]:
!pip install FastAPI